# Logistic Regression Model for Direct Mail Marketing (DMM)

#### This notebook runs on data that was brought over from my work laptop. I made sure to anonymize the data before transferring the data. I have a notebook on that machine where I performed data management, calculated fields, feature engineering. When that notebook is organized well it will join this notebook in the repo. 

#### This notebook stands as the place in which I fit a tree based model for a classification problem. I am attempting to classify probabilities for purchase 'in response' to a direct mail campaign. The target feature is a binary 'yes' 'no' value which indicates if a recipient joined within a specified window of time after the campaign. 

#### This data was highly imbalanced, and some rows were sparse, this required undersampling, oversampling and interpolation methods. However, I am highly suspicious that the results of the model, which gave extremely high accuracy scores, are due to data leakage through my interpolation efforts. I am currently iterating on this notebook, to split the dataset before any interpolation or encoding occurs, to see if this produces a different result. 

In [1]:
# import libraries 

import pandas as pd
import numpy as np 
import sklearn
import geopy
import matplotlib as plt
import plotly
import plotly.express as px
from sklearn.preprocessing import StandardScaler

In [2]:
# read in my csv data 
df = pd.read_csv('/Users/riainfitzsimons/Documents/Projects/Data_Projects/ML_Assignments/Final/safe_master_df.csv')

/var/folders/c2/1d1qzk1s6515n9mmmjl_7b0m0000gn/T/ipykernel_11405/3585020203.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/riainfitzsimons/Documents/Projects/Data_Projects/ML_Assignments/Final/safe_master_df.csv')


In [3]:
# Renaming the columns for ease of interpretation 
df = df.rename(columns={
    'ID': 'id',
    'CITY': 'city',
    'STATE': 'state',
    'FLD04': 'field4',
    'zip': 'zip_code',
    'city': 'city_lower',
    'total_qty': 'total_qty',
    'avg_qty': 'avg_qty',
    'Ev_Date': 'event_date',
    'days_since_last_visit': 'days_since_last',
    'days_between_recent_and_previous': 'days_between_visits',
    'visit_Y/N': 'visit_flag',
    'visit_count': 'visit_count',
    'Sat_Y/N': 'sat_flag',
    'EngagementRating': 'engagement_rating',
    'Target_Y/N': 'target_flag'
})

# Dropping the cont_amt column
df = df.drop(columns=['cont_amt'])

# Displaying the updated dataframe columns
print(df.columns)


Index(['id', 'city', 'state', 'field4', 'zip_code', 'city_lower', 'total_qty',
       'avg_qty', 'event_date', 'days_since_last', 'days_between_visits',
       'visit_flag', 'visit_count', 'sat_flag', 'engagement_rating',
       'target_flag'],
      dtype='object')


In [4]:
# Identify categorical features (object or string types)
categorical_features = df.select_dtypes(include=['object']).columns

# Print the number of unique values for each categorical feature 
for col in categorical_features:
    print(f"Feature '{col}' has {df[col].nunique()} unique values.")


Feature 'city' has 688 unique values.
Feature 'state' has 37 unique values.
Feature 'field4' has 26 unique values.
Feature 'zip_code' has 1594 unique values.
Feature 'city_lower' has 1090 unique values.
Feature 'event_date' has 1839 unique values.
Feature 'target_flag' has 1 unique values.


In [5]:
# debugging target variable

# 'yes'is 1 and anything else is 0 in target_flag
df['target_flag'] = df['target_flag'].apply(lambda x: 1 if x == 'yes' else 0)

# Print the counts of 1 and 0
value_counts = df['target_flag'].value_counts()

print(f"Counts in 'target_flag':\n{value_counts}")


Counts in 'target_flag':
target_flag
0    54560
1      405
Name: count, dtype: int64


In [6]:
# Drop unnecessary columns
df = df.drop(columns=['city_lower', 'event_date'])

# Verify
print("Remaining columns:", df.columns)


Remaining columns: Index(['id', 'city', 'state', 'field4', 'zip_code', 'total_qty', 'avg_qty',
       'days_since_last', 'days_between_visits', 'visit_flag', 'visit_count',
       'sat_flag', 'engagement_rating', 'target_flag'],
      dtype='object')


## encoding categorical features

In [7]:
# Frequency Encoding for high-cardinality features
for col in ['city', 'zip_code']:
    freq = df[col].value_counts(normalize=True)  
    df[col + '_freq'] = df[col].map(freq) 

# Drop the original columns after encoding
df = df.drop(columns=['city', 'zip_code'])

# Verify
print("Columns after frequency encoding:", df.columns)


Columns after frequency encoding: Index(['id', 'state', 'field4', 'total_qty', 'avg_qty', 'days_since_last',
       'days_between_visits', 'visit_flag', 'visit_count', 'sat_flag',
       'engagement_rating', 'target_flag', 'city_freq', 'zip_code_freq'],
      dtype='object')


In [8]:
# One-Hot Encoding for the 'field4' column
df = pd.get_dummies(df, columns=['field4'], drop_first=True)

# Verify the updated columns
print("Columns after one-hot encoding 'field4':", df.columns)


Columns after one-hot encoding 'field4': Index(['id', 'state', 'total_qty', 'avg_qty', 'days_since_last',
       'days_between_visits', 'visit_flag', 'visit_count', 'sat_flag',
       'engagement_rating', 'target_flag', 'city_freq', 'zip_code_freq',
       'field4_Dropped Members 2016', 'field4_Dropped Members 2017',
       'field4_Dropped Members 2018', 'field4_Dropped Members 2019',
       'field4_Dropped Members 2020',
       'field4_Dropped Patrons 2020 and earlier',
       'field4_Dropped Patrons 2021', 'field4_Dropped Patrons 2022',
       'field4_Dropped Patrons 2023', 'field4_Dropped Patrons 2024',
       'field4_Lapsed & Dropped Members 2022',
       'field4_Lapsed & Dropped Members 2023',
       'field4_Lapsed & Dropped Members 2024',
       'field4_Ticket Buyers - Christian Dior',
       'field4_Ticket Buyers - Education Prog',
       'field4_Ticket Buyers - First Saturday',
       'field4_Ticket Buyers - GA 2023', 'field4_Ticket Buyers - GA 2024',
       'field4_Ticket Buye

In [10]:
# Count the occurrences of each state
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'count']

# Create the bar chart
fig = px.bar(
    state_counts,
    x='state',
    y='count',
    title='Distribution of States',
    labels={'state': 'State', 'count': 'Count'},
    text='count'
)

# Customize the layout
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_tickangle=-45,
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Show the plot
fig.show()


In [11]:
# Create a binary column: 1 for 'NY', 0 for all other states
df['is_new_york'] = df['state'].apply(lambda x: 1 if x == 'NY' else 0)

# Drop the original 'state' column
df = df.drop(columns=['state'])

# Verify the transformation
print(df['is_new_york'].value_counts())


is_new_york
1    54348
0      617
Name: count, dtype: int64


In [12]:
# List of numeric columns to scale
numeric_cols = ['total_qty', 'avg_qty', 'days_since_last', 'days_between_visits', 
                'engagement_rating', 'visit_count', 'city_freq', 'zip_code_freq']

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [13]:
# Convert boolean columns to integers
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [14]:
# Fill missing values in 'engagement_rating' with the column mean
df['engagement_rating'] = df['engagement_rating'].fillna(df['engagement_rating'].mean())``

# Verify there are no more missing values in 'engagement_rating'
print(f"Number of null values in 'engagement_rating': {df['engagement_rating'].isnull().sum()}")


Number of null values in 'engagement_rating': 0


## undersampling

### Handling Missing Data with Row-Wise Missingness

I noticed that some rows in the dataset have a high proportion of missing data. To address this, I plan to calculate the **missingness percentage** for each row and selectively under-sample rows with excessive null values. 

The goal is to reduce noise while preserving enough data for analysis. By plotting the distribution of missingness and reviewing summary statistics, I'll decide on a threshold for how much missing data is acceptable. Rows exceeding this threshold will be under-sampled to avoid overrepresentation of incomplete data, and the rest will be kept intact. 

This approach helps maintain the balance between dataset quality and size while ensuring that critical patterns are not lost.


In [15]:
# Print the number of nulls for each feature
null_counts = df.isnull().sum()

print("Number of null values for each feature:")
print(null_counts)

Number of null values for each feature:
id                                             0
total_qty                                   2594
avg_qty                                    23367
days_since_last                            23367
days_between_visits                        32932
visit_flag                                     0
visit_count                                    0
sat_flag                                       0
engagement_rating                              0
target_flag                                    0
city_freq                                      0
zip_code_freq                               1717
field4_Dropped Members 2016                    0
field4_Dropped Members 2017                    0
field4_Dropped Members 2018                    0
field4_Dropped Members 2019                    0
field4_Dropped Members 2020                    0
field4_Dropped Patrons 2020 and earlier        0
field4_Dropped Patrons 2021                    0
field4_Dropped Patrons 2022  

In [16]:
# Check if rows with null 'zip_code_freq' have a positive 'target_flag' value
null_zip_positive_target = df[(df['zip_code_freq'].isnull()) & (df['target_flag'] == 1)]

# Count 
num_null_zip_positive_target = null_zip_positive_target.shape[0]

print(f"Number of rows with null 'zip_code_freq' and a positive 'target_flag': {num_null_zip_positive_target}")


Number of rows with null 'zip_code_freq' and a positive 'target_flag': 0


### there are no rows that have missing zip data that are in the minority class, so im opting to remove those rows

In [17]:
# Remove rows with null values in 'zip_code_freq' and save to a new dataframe
df_undersampled = df[df['zip_code_freq'].notnull()]

# Verify the result
print(f"Dataset size after removing rows with null 'zip_code_freq': {df_undersampled.shape[0]} rows")


Dataset size after removing rows with null 'zip_code_freq': 53248 rows


In [32]:
# Calculate row-wise missingness
df['missingness'] = df.isnull().sum(axis=1) / df.shape[1]

# Plot the distribution of missingness using Plotly Express
fig = px.histogram(
    df,
    x='missingness',
    nbins=30,
    title='Distribution of Row-Wise Missingness',
    labels={'missingness': 'Proportion of Missing Features'},
    template='plotly_white'
)

fig.update_layout(
    xaxis_title='Proportion of Missing Features',
    yaxis_title='Number of Rows',
    title_font_size=20,
    xaxis_title_font_size=17,
    yaxis_title_font_size=17,
    bargap=0.1
)

# Show the plot
fig.show()


In [19]:
# Set the threshold for high missingness
missingness_threshold = 0.06  # 6% missingness

# Split rows into high and low missingness groups
high_missingness_rows = df[df['missingness'] > missingness_threshold]
low_missingness_rows = df[df['missingness'] <= missingness_threshold]

# Filter high missingness rows to retain only majority class (target_flag = 0)
high_missingness_majority = high_missingness_rows[high_missingness_rows['target_flag'] == 0]

# Under-sample high-missingness rows (only from majority class)
sampling_fraction = 0.3  # Retain 30% of high-missingness majority class rows
high_missingness_sample = high_missingness_majority.sample(frac=sampling_fraction, random_state=42)

# Combine low-missingness rows with sampled high-missingness rows
df_undersampled = pd.concat([low_missingness_rows, high_missingness_sample], axis=0)

# Drop the 'missingness' helper column
df_undersampled = df_undersampled.drop(columns=['missingness'])

# Verify the result
print(f"Original dataset size: {df.shape[0]} rows")
print(f"Cleaned dataset size: {df_undersampled.shape[0]} rows")



Original dataset size: 54965 rows
Cleaned dataset size: 38591 rows


### complete the interpolation process

In [20]:
# List of features with null values to impute
features_with_nulls = ['total_qty', 'avg_qty', 'days_since_last', 'days_between_visits']

# Impute nulls by the mean of their 'zip_code_freq' group
for feature in features_with_nulls:
    df_undersampled[feature] = df_undersampled.groupby('zip_code_freq')[feature].transform(
        lambda x: x.fillna(x.mean())
    )

# Verify if there are remaining nulls
remaining_nulls = df_undersampled[features_with_nulls].isnull().sum()
print("Remaining null values after imputation:\n", remaining_nulls)

Remaining null values after imputation:
 total_qty              522
avg_qty                522
days_since_last        522
days_between_visits    522
dtype: int64


In [21]:
# For any remaining nulls, fallback to the overall mean of the column
for feature in features_with_nulls:
    df_undersampled[feature] = df_undersampled[feature].fillna(df_undersampled[feature].mean())

# Final check for null values
final_nulls = df_undersampled[features_with_nulls].isnull().sum()
print("Final null values after fallback imputation:\n", final_nulls)

Final null values after fallback imputation:
 total_qty              0
avg_qty                0
days_since_last        0
days_between_visits    0
dtype: int64


In [22]:
# Print the number of nulls for each feature
null_counts = df_undersampled.isnull().sum()

print("Number of null values for each feature:")
print(null_counts)

Number of null values for each feature:
id                                           0
total_qty                                    0
avg_qty                                      0
days_since_last                              0
days_between_visits                          0
visit_flag                                   0
visit_count                                  0
sat_flag                                     0
engagement_rating                            0
target_flag                                  0
city_freq                                    0
zip_code_freq                              522
field4_Dropped Members 2016                  0
field4_Dropped Members 2017                  0
field4_Dropped Members 2018                  0
field4_Dropped Members 2019                  0
field4_Dropped Members 2020                  0
field4_Dropped Patrons 2020 and earlier      0
field4_Dropped Patrons 2021                  0
field4_Dropped Patrons 2022                  0
field4_Dropped Patro

In [23]:
# Remove rows with null values in 'zip_code_freq'
df_undersampled = df_undersampled[df_undersampled['zip_code_freq'].notnull()]

# Verify the result
print(f"Dataset size after removing rows with null 'zip_code_freq': {df_undersampled.shape[0]} rows")


Dataset size after removing rows with null 'zip_code_freq': 38069 rows


### oversampling 

### first i need to split dataset 

In [24]:
# Check class distribution of the target variable
class_counts = df_undersampled['target_flag'].value_counts()
print(f"Class distribution in 'target_flag':\n{class_counts}")


Class distribution in 'target_flag':
target_flag
0    37721
1      348
Name: count, dtype: int64


In [25]:
# %% [markdown]
# ## Apply SMOTE to `df_undersampled`

# %%
from imblearn.over_sampling import SMOTE

# Separate features (X) and target (y) in df_undersampled
X_undersampled = df_undersampled.drop(columns=['target_flag'])
y_undersampled = df_undersampled['target_flag']

# Initialize SMOTE
smote = SMOTE(sampling_strategy=0.015, random_state=42)

# Apply SMOTE to balance the classes in df_undersampled
X_resampled, y_resampled = smote.fit_resample(X_undersampled, y_undersampled)

# Verify the new class distribution after SMOTE
print("New class distribution after SMOTE:\n", y_resampled.value_counts())


New class distribution after SMOTE:
 target_flag
0    37721
1      565
Name: count, dtype: int64


In [26]:
# %%
# Now, we can split the resampled data into training and test sets (optional, if needed)
# This step can be skipped if SMOTE was already applied to the entire dataset
from sklearn.model_selection import train_test_split

X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(
    X_resampled, y_resampled, test_size=0.2, stratify=y_resampled, random_state=42)

# Proceed with training the model on the resampled training set
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

rf_model = RandomForestClassifier(random_state=42, class_weight="balanced")
rf_model.fit(X_train_resampled, y_train_resampled)

# Evaluate on the test set
y_pred_rf = rf_model.predict(X_test_resampled)
y_pred_probs_rf = rf_model.predict_proba(X_test_resampled)[:, 1]

# Print classification report
print("Random Forest Classification Report:")
print(classification_report(y_test_resampled, y_pred_rf))

# Calculate and print ROC-AUC score
roc_auc_rf = roc_auc_score(y_test_resampled, y_pred_probs_rf)
print(f"Random Forest ROC-AUC Score: {roc_auc_rf}")



Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7545
           1       0.99      0.69      0.81       113

    accuracy                           1.00      7658
   macro avg       0.99      0.85      0.91      7658
weighted avg       1.00      1.00      0.99      7658

Random Forest ROC-AUC Score: 0.9871954116011893


In [27]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on the training set
cv_scores = cross_val_score(rf_model, X_train_resampled, y_train_resampled, cv=5, scoring='roc_auc')

print(f"Cross-Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean Cross-Validation ROC-AUC: {cv_scores.mean()}")


Cross-Validation ROC-AUC Scores: [0.99079504 0.96514654 0.97741107 0.98912731 0.98443984]
Mean Cross-Validation ROC-AUC: 0.981383961440175


In [28]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', {0: 1, 1: 5}]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    n_jobs=-1,
    verbose=2
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Print best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best ROC-AUC Score:", grid_search.best_score_)


Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.4s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.4s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.6s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.0s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.6s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.1s
[CV] END class_weight=balanced, max_depth=10, min_samples_leaf=1, min